In [ ]:
import mindspore
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindnlp.peft import PeftModel

# 设置设备（Ascend/GPU/CPU）
mindspore.set_device('Ascend')
!npu-smi info

[WARNING] ME(16575:281472988614832,MainProcess):2025-09-22-18:46:39.782.042 [mindspore/run_check/_check_version.py:324] MindSpore version 2.5.0 and Ascend AI software package (Ascend Data Center Solution)version 7.7 does not match, the version of software package expect one of ['7.5', '7.6']. Please refer to the match info on: https://www.mindspore.cn/install
[WARNING] ME(16575:281472988614832,MainProcess):2025-09-22-18:46:39.784.697 [mindspore/run_check/_check_version.py:402] Can not find the tbe operator implementation(need by mindspore-ascend). Please check whether the Environment Variable PYTHONPATH is set. For details, refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(16575:281472988614832,MainProcess):2025-09-22-18:46:39.785.480 [mindspore/run_check/_check_version.py:342] MindSpore version 2.5.0 and "te" wheel package version 7.7 does not match. For details, refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(16575

+------------------------------------------------------------------------------------------------+
| npu-smi 23.0.6                   Version: 23.0.6                                               |
+---------------------------+---------------+----------------------------------------------------+
| NPU   Name                | Health        | Power(W)    Temp(C)           Hugepages-Usage(page)|
| Chip                      | Bus-Id        | AICore(%)   Memory-Usage(MB)  HBM-Usage(MB)        |
+===========================+===============+====================================================+
| 0     910B2               | OK            | 94.4        49                0    / 0             |
| 0                         | 0000:C1:00.0  | 0           0    / 0          3379 / 65536         |
+===========================+===============+====================================================+
+---------------------------+---------------+----------------------------------------------------+
| NPU     

In [ ]:
# ========= 1. 加载基座模型 =========
base_model_path = "./Qwen2.5-7B"   # 你的基座模型路径
adapter_model_path = "./output/qwen2.5-7B/checkpoint-25/adapter_model"  # LoRA 权重路径

print(">>> 加载基座模型...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    ms_dtype=mindspore.bfloat16
)

print(">>> 加载 tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# ========= 2. 加载 LoRA 微调权重 =========
print(">>> 加载 LoRA adapter 权重...")
model = PeftModel.from_pretrained(base_model, adapter_model_path)

Qwen2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


>>> 加载基座模型...


[WARNING] DEVICE(16575,fffe933b91e0,python):2025-09-22-18:51:30.570.788 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:147] CheckVmmDriverVersion] Open file /etc/ascend_install.info failed.
[WARNING] DEVICE(16575,fffe933b91e0,python):2025-09-22-18:51:30.570.994 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:186] CheckVmmDriverVersion] Driver version is less than 24.0.0, vmm is disabled by default, drvier_version: 23.0.6
Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.66s/it]


>>> 加载 tokenizer...
>>> 加载 LoRA adapter 权重...


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "如何联系本科生就业办公室咨询相关事宜？"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...
....
===== 模型输出 =====

system
You are a helpful assistant.
user
如何联系本科生就业办公室咨询相关事宜？
assistant
81891632本科生就业办公室


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "校园乐跑的有效跑步标准是什么？"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
校园乐跑的有效跑步标准是什么？
assistant
男生有效跑步标准：单次跑步里程达到3km，配速在3～9min/km范围内。  
女生有效跑步标准：单次跑步里程达到2km，配速在3～9min/km范围内。  
（每天6:00～23:00完成，体育课内跑步无效）


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "新生入学时是否需要参加开学考试？"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
新生入学时是否需要参加开学考试？
assistant
根据校规内容，新生入学时需要参加开学考试。具体考试安排和要求请以学校正式通知为准。


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "我校的校训是什么"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
如果我想查看奖学金、助学金的相关通知，应该关注哪个公众号？
assistant
西电学生奖助


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "学校有几个菜鸟驿站，分别在哪里?"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
学校有几个菜鸟驿站，分别在哪里?
assistant
学校有以下4个菜鸟驿站：

1. 菜鸟驿站（老综合楼靠北门边）
2. 菜鸟驿站（老综合楼靠南门边）
3. 菜鸟驿站（老综合楼香八度二楼）
4. 菜鸟驿站（丁香12号楼后）


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "西安电子科技大学南校区地址在哪?"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
西安电子科技大学南校区地址在哪?
assistant
西安市太白南路2号


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "开学考的成绩是否会影响实验班的分流？?"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
开学考的成绩是否会影响实验班的分流？?
assistant
是的，开学考的成绩会影响实验班的分流。如果你高考直接进入了实验班，开学考的成绩将决定你的班级分流。实验班相当于一次转专业的机会，想转专业的同学可以关注开学考。更多详情可以参考：https://mobile.xidian.edu.cn/info/1228/57686.htm


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "在哪里可以找到京东快递点？?"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
在哪里可以找到京东快递点？?
assistant
京东快递点位于老综合楼靠北门边缘。


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "我校的校训是什么?"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
我校的校训是什么?
assistant
厚德、求真、砺学、笃行


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "考试作弊会面临什么处分?"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
考试作弊会面临什么处分?
assistant
根据学生手册规定，考试作弊将视情节轻重给予记过及以上处分。具体包括以下行为：
1. 代替他人或让他人代替自己参加考试
2. 组织作弊
3. 使用通讯设备或其他器材作弊
4. 向他人出售考试试题或答案牟取利益
5. 其他严重作弊行为
6. 扰乱考场秩序
7. 故意扰乱考场秩序
8. 抢夺、窃取试卷或答案
9. 故意销毁试卷、答卷或考试材料
10. 拒绝、妨碍考试工作人员履行管理职责
11. 威胁、侮辱、诽谤、诬陷考试工作人员


In [ ]:
# ========= 3. 构造输入 =========
prompt_messages = [
    {"role": "user", "content": "学校本科生宿舍的基本配置有哪些?"}
]

# 转换成模型输入格式
prompt_text = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer(prompt_text, return_tensors="ms")

# ========= 4. 推理生成 =========
print(">>> 开始生成...")
outputs = model.generate(
    **inputs,
    max_length=512,
    do_sample=True,    # 采样模式（更有创意）
    top_p=0.9,         # nucleus sampling
    temperature=0.8    # 控制多样性
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n===== 模型输出 =====\n")
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


>>> 开始生成...

===== 模型输出 =====

system
You are a helpful assistant.
user
学校本科生宿舍的基本配置有哪些?
assistant
本科生宿舍基本配置包括：上床下桌，配备电话、暖气、空调和网络，提供热水洗浴服务。宿舍区域分布在竹园、海棠、丁香等公寓，各区域均设有学生餐厅。
